# LITERATURE IN THE AGE OF ARTIFICIAL INTELLIGENCE FINAL

1. All references listed in cells where they are used.

# Dependencies

In [1]:
# https://github.com/tensorflow/agents/blob/master/docs/tutorials/2_environments_tutorial.ipynb

!pip install tf-agents
!pip install 'gym==0.10.11'
!pip install fake-useragent

import abc
import tensorflow as tf
import numpy as np
import random
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

     |████████████████████████████████| 911kB 2.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Found existing installation: gin-config 0.3.0
    Uninstalling gin-config-0.3.0:
      Successfully uninstalled gin-config-0.3.0
     |████████████████████████████████| 1.5MB 2.7MB/s 
  Created wheel for gym: filename=gym-0.10.11-cp36-none-any.whl size=1588313 sha256=72e2dfe99578ae2bfdaa4c6db7bf3a1670555b9eb1e1e6835ab5a939642d9a6f
  Stored in directory: /root/.cache/pip/wheels/7b/eb/1f/22c4124f3c64943aa0646daf4612b1c1f00f27d89b81304ebd
Successfully built gym
  Found existing installation: gym 0.17.1
    Uninstalling gym-0.17.1:
      Successfully uninstalled gym-0.17.1
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=67f4869535c2ca80ade177627a0aafc3ab699c8aa5ced39596382c234e275a4d
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


# Environment Setup

In [0]:
'''
    # The poem as three arrays representing the following:
    # The poem consists of 12 sentences.
    
    first_word : a 1-d array where the index corresponds to the (sentence number - 1). The value of first_word[10] is 'Lest' from the 11th sentence.
    second_word : a 2-d array where the first index corresponds to the (sentence number - 1).
                                    the second index corresponds to one of (x-1) number of words that can be selected in that sentence.
                                    The value of second_word[10][4] is the 5th word (' Mind ') of the 11th sentence.
    third_word : a 1-d array where the index corresponds to the (sentence number - 1). The value of third_word[10] is 'Be Destroyed from the 11th sentence.

    note : Ending of 7th line has been changed from 'and g-d' to 'and god'
'''
first_word = ['from', 'come', 'from', 'come', 'the', 'love', 'the', 'seek', 'what', 'with', 'lest', 'where']

second_word = [
            [' night ', ' fog ', ' clash ', ' frost ', ' wind ', ' sea ', ' heat ', ' south ', ' east ', ' west ', ' north ', ' sun ', ' fire '],
            [' day ', ' blaze ', ' bloom ', ' snow ', ' peace ', ' land ', ' flash ', ' warmth ', ' heat ', ' joy ', ' cool ', ' light ', ' flames '],
            [' ache ', ' pain ', ' shame ', ' fear ', ' war ', ' oh ', ' cross ', ' stress ', ' scorn ', ' hurt ', ' shock ', ' guile ', ' scold '],
            [' joy ', ' charm ', ' fame ', ' ease ', ' wins ', ' truce ', ' gains ', ' peace ', ' jokes ', ' games ', ' rest ', ' mild '],

            [' moon ', ' beam ', ' smoke ', ' buck ', ' fish ', ' gold ', ' pearl ', ' tree ', ' flame ', ' stork ', ' frog ', ' lamb ox ', ' all '],
            [' sheen ', ' straw ', ' steam ', ' peak ', ' flood ', ' glow ', ' foam ', ' fruit ', ' ash ', ' roof ', ' pool ', ' field ', ' wheat '],
            [' guard ', ' man ', ' work ', ' strain ', ' art ', ' play ', ' ship ', ' mouth ', ' prince ', ' rage ', ' care ', ' greed ', ' faith '],
            [' goal ', ' sleep ', ' praise ', ' prize ', ' luck ', ' strife ', ' port ', ' kiss ', ' throne ', ' death ', ' shroud ', ' gold ', ' thanks '],

            [' good ', ' strong ', ' hard ', ' right ', ' long ', ' great ', ' white ', ' one ', ' yea ', ' air ', ' fire ', ' high wide ', ' should '],
            [' care ', ' bad ', ' weak ', ' plain ', ' bent ', ' broad ', ' small ', ' black ', ' three ', ' nay ', ' earth ', ' flood ', ' deep '],
            [' pluck ', ' love ', ' wise ', ' wit ', ' mind ', ' soul ', ' friend ', ' joy ', ' grace ', ' fame ', ' peace ', ' play ', ' praise '],
            [' fear ', ' hate ', ' lies ', ' wine ', ' meat ', ' flesh ', ' foe ', ' woe ', ' shame ', ' angst ', ' strife ', ' pain ', ' rule ']
         ]

third_word = ['and plagues', 'and dread', 'and fraud', 'morning rays', 'at play', 'and bread', 'and god', 'to say', 'be named', 'to avoid', 'be destroyed', 'the day']

In [4]:
# Quick Test to ensure embedding is loaded.

select = []

for x in range(len(first_word)):
  random.seed()

  selection = random.randint(0,len(second_word[x])-1)
  select.append(selection)

  search_term = first_word[x] + second_word[x][selection] + third_word[x]
  print(search_term)

print (select)

from fog and plagues
come blaze and dread
from pain and fraud
come joy morning rays
the fish at play
love sheen and bread
the greed and god
seek shroud to say
what one be named
with earth to avoid
lest friend be destroyed
where meat the day
[1, 1, 1, 0, 4, 0, 11, 10, 7, 10, 6, 4]


In [0]:
'''
USED TO RETURN GOOGLE SEARCH TOTALS
'''
def pyGoogleSearch(word,agent):
    header = {'User-Agent':str(agent.chrome)}
    address = 'http://www.google.com/search?q='
    newword = address + word
    page = requests.get(newword, headers=header)
    soup = BeautifulSoup(page.content, 'html.parser')
    res = soup.find("div", {"id": "result-stats"})
    try:
      final = res.text.replace(",", "").split()[1]
      final = int(final)
      return final
    except:
      return 0

In [0]:
'''
TEST AREA PRIOR TO DEVELOPMENT OF REINFORCEMENT LEARNING ENVIRONMENT
'''
ua = UserAgent()

# 
sec_word_array = []
scores = []
reward = 0

for x in range(len(first_word)):
  random.seed()

  # Load search term
  selection = random.randint(0,len(second_word[x])-1)
  sec_word_array.append(selection)
  search_term = "\"" +  first_word[x] + second_word[x][selection] + third_word[x] + "\""

  # return results for searching line
  score = pyGoogleSearch(search_term, ua)
  scores.append(score)
  
  # add total number of results
  reward += score
  print("Search Term: %s -- Results: %d" %(search_term, score))

print( sec_word_array )
print( scores )
print( reward )

In [0]:
'''
INITIAL REINFORCEMENT LEARNING ENVIRONMENT
'''

class Kuhlmann(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(2,10), dtype=np.int32, minimum=0, name='observation')
    self._state = array_spec.BoundedArraySpec(
        shape=(2,10), dtype=np.int32, minimum=0, name='state')
    self._episode_ended = False
    self.ua = UserAgent()
  
  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):
    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      #generate poem
      sec_word_array = []
      scores = []
      total_reward = 0

      for x in range(len(first_word)):
        random.seed()

        # Load search term
        selection = random.randint(0,len(second_word[x])-1)
        sec_word_array.append(selection)
        search_term = "\"" +  first_word[x] + second_word[x][selection] + third_word[x] + "\""

        # return results for searching line
        score = pyGoogleSearch(search_term, self.ua)
        scores.append(score)
        
        # add total number of results
        total_reward += score
        print("Search Term: %s -- Results: %d" %(search_term, score))

      print( sec_word_array )
      print( scores )
      print( total_reward )
      
      self._state = [sec_word_array, scores]
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended:
      reward = total_reward
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [0]:
'''
FOR TESTING PURPOSES
'''
environment = Kuhlmann()
utils.validate_py_environment(environment, episodes=5)

In [8]:
'''
FOR TESTING PURPOSES
'''

generate_poem = np.array(0, dtype=np.int32)
record_stats = np.array(1, dtype=np.int32)

environment = Kuhlmann()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(5):
  time_step = environment.step(generate_poem)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(record_stats)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([9], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([11], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([15], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([17], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-21., dtype=float32), discount=array(0., dtype=float32), observation=array([26], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-21., dtype=float32), discount=array(0., dtype=float32), 

# MANUAL TESTING

This section was created after manually obtaining total search results.


In [51]:
google_results = [
                  [0,0,0,1,0,0,0,0,0,0,0,0,10],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,1,0,24500,5,5,0,1,0,0,0,4,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [27800,0,1,1,22400,0,2,38100,1,6,6,0,3],
                  [0,0,0,0,0,0,0,15900,0,0,0,0,5],
                  [121000,6710000,6840000,56300,2080000,289000,308000,2,278000,86000,7,46100,6290000],
                  [0,0,1,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,1550000],
                  [4980000,4,0,19000,0,1,0,389000,4,1,194000,1,4],
                  [0,0,0,0,0,0,0,0,0,0,3,0,0],
                  [1,0,0,0,0,0,1,1,0,0,0,0,0]
                  ]
np_arr = np.array(google_results)
np_arr.shape

(12, 13)

In [0]:
def reconstruct(second_word_array):
  for index, val in enumerate(second_word_array):
    if index % 4 == 0:
      print(" ")
    print(first_word[index]+second_word[index][val]+third_word[index])
  
  print (" ")
  print ("all things do change; all things do love; all things are locked in hate")
  if np.sum(second_word_array) % 2 == 0:
    print('whoever fully thinks this through holds the key to man\'s estate')
  else:
    print('holds the key to man\'s estate whoever fully thinks this through')


In [55]:
'''
  SET RESULTS THRESHOLD BY ALTERING FIRST VARIABLE 'threshold' BELOW.
'''

threshold = 0

max_val_output = []
np_gr = np.array(google_results)
for row in np_gr:
  random.seed()
  selection = random.randint(0,len(second_word[x])-1)
  if row.sum() > threshold:
    max_val_output.append(row.argmax(axis=0))
  else:
    max_val_output.append(selection)

reconstruct(max_val_output)
print(max_val_output)

 
from sun and plagues
come warmth and dread
from fear and fraud
come charm morning rays
 
the tree at play
love fruit and bread
the work and god
seek kiss to say
 
what should be named
with care to avoid
lest love be destroyed
where wine the day
 
all things do change; all things do love; all things are locked in hate
holds the key to man's estate whoever fully thinks this through
[11, 7, 3, 1, 7, 7, 2, 7, 12, 0, 1, 3]
